## Import dependencies

In [ ]:
import pandas as pd
import requests
import json
import random


from helper_function.notebook_helpers import show_vehicle_routes, get_minutes_from_datetime
from helper_function.map_helpers import get_map_by_vehicle

from cuopt_thin_client import CuOptServiceClient

Finally, suppose you are working at a company for grocery store fridge installation and maintenance. Once again, you are working with the same grocery stores. Some of the grocery stores are new so they have put in a request to install fridges. Some of the stores already have fridges but request some sort of maintenance. Given input data about stores' service requests and the available fleet of vehicles, it is your job to calculate the route for each vehicle such that all service requests are fulfilled while minimizing vehicles' travel time and cost. In this notebook, we will walk through the data preprocessing steps needed in order to utilize cuOpt for this use case. 

## Read input data from CSV files

For a Dispatch Optimization problem, we need 3 datasets with the following features:

- Depots
    - Name
    - Location
    - Start and end time (operation hours)

- Orders
    - Location
    - Start and end time (customer indicated time window)
    - Demand (service type- either install or maintenance)

- Vehicles
    - Name/ID Number
    - Start and end depot name
    - Start and end time (vehicle/driver shift hours)
    - Capacity (in this problem is given in time- how long a driver can work for)
    - Vehicle skills (whether this driver can provide install or maintenance service)



You may have additional features depending on the problem at hand.

In [ ]:
depots_df = pd.read_csv('data/depots_do.csv')
orders_df = pd.read_csv('data/orders_do.csv')
vehicles_df = pd.read_csv('data/vehicles_do.csv')

In [ ]:
n_depots = len(depots_df.index)
n_orders = len(orders_df.index)
n_vehicles = len(vehicles_df.index)

n_loc_total = n_orders + n_depots

In [ ]:
locations_df = (pd.concat([depots_df[["Name","Longitude","Latitude"]], orders_df[["Name","Longitude","Latitude"]]], ignore_index=True)).reset_index()

# Create cost matrices

### Cost Matrix - Distance 

For our primary <code style="background:lightgreen;color:black">cost_matrix</code>, we will use travel distance. In practical applications, you can integrate this to a third-party map data provider like Esri or Google Maps to get live traffic data and run dynamic/real-time re-routing using cuOpt.

We've already created this cost matrix using Google API and saved it as a csv so you can easily read it from the csv file.

If you want to build the cost matrix on your own, or if you are working with your own data, refer to the [LMD notebook](../01_LMD_workflow/LMD_workflow.ipynb)


In [ ]:
import pandas as pd

df = pd.read_csv('data/cost_matrix_distance.csv', header=None)
cost_matrix_distance = df.astype(int).values.tolist()

### Cost Matrix - Time

Next, let's create the <code style="background:lightgreen;color:black">travel_time_matrix</code>.
We already have travel time data from Google Maps API (this data is in the 'durations in sections' column in our output dataframe. However, let's take a look at using a different tool for this. We will use OSRM to calculate the travel time in minutes between each two pairs of locations which. 

[OSRM](https://project-osrm.org/) is a free and open and open source routing engine, which we will use for route mapping and visualization later on. 


In [ ]:
latitude = locations_df.Latitude.to_numpy()
longitude = locations_df.Longitude.to_numpy()
    
locations=""
n_orders = len(locations_df)
for i in range(n_orders):
    locations = locations + "{},{};".format(longitude[i], latitude[i])
r = requests.get("http://router.project-osrm.org/table/v1/car/"+ locations[:-1])
routes = json.loads(r.content)
    
# OSRM returns duration in seconds. Here we are converting to minutes
for i in routes['durations']:
    i[:] = [x / 60 for x in i]
    
coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
time_matrix_df = pd.DataFrame(routes['durations'])
time_matrix = time_matrix_df.values.tolist()

### Set fleet data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

<code style="background:lightgreen;color:black">vehicle_locations</code> is a list of the start and end location of the vehicles. For example, a vehicle that starts and ends in depot 1 which is the location at index 0 would have the vehicle location of [0,0]. While each vehicle has an assigned location, in this use case, drivers may start and end their shift wherever they'd like. For example, they might wake up at home in the morning and go directly to their first task. Similarly, at the end of the day, they might finish their last task and go straight home without stopping in their depot. To represent this, we pass an array of booleans for <code style="background:lightgreen;color:black">skip_first_trips</code> and <code style="background:lightgreen;color:black">skip_last_trips</code>, where the value is True for all vehicles

In [ ]:
depot_names_to_indices_dict = {locations_df["Name"].values.tolist()[i]: i for i in range(n_depots)}
vehicle_locations = vehicles_df[["assigned_depot","assigned_depot"]].replace(depot_names_to_indices_dict).values.tolist()

In [ ]:
skip_first_trips = [True]*n_vehicles
drop_return_trips = [True]*n_vehicles

<code style="background:lightgreen;color:black">vehicle_time_windows</code> is a list of the integer representation of the operating time of each vehicle. Equivalently, the shift of each vehicle driver. We convert the UTC timestamp to epoch time (integer representation in minutes).

In [ ]:
vehicle_time_windows = pd.concat((vehicles_df['vehicle_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['vehicle_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

<code style="background:lightgreen;color:black">vehicle_max_times</code> is the maximum length of a shift a driver should work. For example, a driver might be available to work for 9 hours in a day but a shift should not exceed 6 hours, such that the driver will work 6 out of these 9 hours.

In [ ]:
vehicle_max_times = vehicles_df['vehicle_capacity'].values.tolist()

### Set task data


Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

<code style="background:lightgreen;color:black">task_locations</code> is the locations where customers have requested service. 

In [ ]:
task_locations = locations_df.index.tolist()[n_depots:]

<code style="background:lightgreen;color:black">task_time_windows</code> is the list of integer representation of the customer indicated time window in which the service provider can come to deliver the requested service. 

In [ ]:
task_time_windows = pd.concat((orders_df['order_start_time'].apply(get_minutes_from_datetime).to_frame(), orders_df['order_end_time'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

<code style="background:lightgreen;color:black">demand_time</code> is the list of the length of time it takes for each service to be fulfilled. We let install service be 60 minutes, and maintenance service be 120 minutes.

In [ ]:
demand_service_time = {"install":60, "maintenance":90}
demand_time = orders_df["service"].replace(demand_service_time).tolist()

<code style="background:lightgreen;color:black">order_vehicle_match</code> is a list of dictionaries that map which vehicles can provide the service requested in each location. Some vehicles can provide install service, some can provide maintenance service, and some can do both. For a task that has requested install service, we want to assign all the vehicles that can fulfill this type of request.

In [ ]:
install_tech_ids = vehicles_df['install_service'][vehicles_df['install_service']==1].index.values.tolist()
maintenance_tech_ids = vehicles_df['maintenance_service'][vehicles_df['maintenance_service']==1].index.values.tolist()

In [ ]:
vehicle_match_list = []

for i in range(len(orders_df['service'].index)):
    if orders_df['service'][i] == 'install':
        vehicle_match_list.append({"order_id": i, "vehicle_ids": install_tech_ids})
    if orders_df['service'][i] == 'maintenance':
        vehicle_match_list.append({"order_id": i, "vehicle_ids": maintenance_tech_ids}) 


### Set Solver configuration

Before we send our data to the cuOpt solver, we will add two configuration settings.

<code style="background:lightgreen;color:black">time_limit</code> is the maximum time allotted to find a solution. This depends on the user, who has the flexibility of setting a higher time‑limit for better results. The cuOpt solver does not interrupt the initial solution. So if the user specifies a shorter time than it takes for the initial solution, the initial solution is returned when it is computed.

In [ ]:
# Set the time limit 

time_limit = 1.0

## Save data in a dictionary

Here, we take all the data we have prepared so far and save it to one dictionary. This includes the cost matrices, task data, fleet data, and solver config. This is all the data that cuOpt needs to solve our dispatch optimization problem. 

In [ ]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "data": {
            "0": cost_matrix_distance
        }
    },
    "travel_time_matrix_data": {
        "data": {
            "0": time_matrix
        }
    },
    "task_data": {
        "task_locations": task_locations,
        "task_time_windows": task_time_windows,
        "service_times": demand_time,
        "order_vehicle_match": vehicle_match_list,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "skip_first_trips" : skip_first_trips,
        "drop_return_trips": drop_return_trips,
        "vehicle_max_times": vehicle_max_times,
        "vehicle_time_windows": vehicle_time_windows,
    },
    "solver_config": {
        "time_limit": time_limit,
    }
}

## Create a Service Client Instance

Now that we have prepared all of our data, we can establish a connection to the cuOpt service. 

In the cell below, there is a place to paste a client SAK (Starfleet API Key) which you can generate from the NGC console. In this lab, you do not need to provide it.

Here, we create an instance of the cuOpt Service Client to establish a connection. 


In [ ]:
# Currently this notebook works with spoofed SAK and FUNCTION ID, but users need to use their own SAK and FUNCTION ID if
# they are going to run this notebook in their local environment

cuopt_client_sak = "<YOUR CLIENT SAK>"

cuopt_service_client = CuOptServiceClient(
    sak=cuopt_client_sak,
    function_id="<FUNCTION_ID_OBTAINED_FROM_NGC>"
    )

## Send data to the cuOpt service and get the routes

When using the cuOpt Managed Service, we send all the data in a single call and wait for the response.

In [ ]:

# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]
if "solver_response" in solver_resp:
    solver_resp = solver_resp["solver_response"]
else:
    # For our purposes here if we get an infeasible response,
    # we treat it as a successful solution
    solver_resp = solver_resp["solver_infeasible_response"]
    solver_resp["status"] = 0
    print("Infeasible solution found!")

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

# Visualize the routes

In this example, not all vehicles are dispatched. It is possible that vehicle 0 is not dispatched but vehicle 1 is.  

In the drop down menu below, you can select different vehicle ID's to see if they are dispatched. If they are, we print their assigned route on a map. 

Generating a route and map uses third party tools and takes about 30 seconds to run. 


In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options = list(vehicles_df.index.values),
    description='Vehicle ID:',
)

def on_change(value):
    if str(value) in list(solver_resp['vehicle_data'].keys()):
        if len(solver_resp["vehicle_data"][str(value)]['route']) == 1:
            l = solver_resp["vehicle_data"][str(value)]['route'][0]
            solver_resp["vehicle_data"][str(value)]['route'] = [l,l]
        curr_route_df = pd.DataFrame(solver_resp["vehicle_data"][str(value)]['route'], columns=["stop_index"])
        curr_route_df = pd.merge(curr_route_df, locations_df, how="left", left_on=["stop_index"], right_on=["index"])
        display(get_map_by_vehicle(curr_route_df))        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)

## Objective Functions

<code style="background:lightgreen;color:black">variance_route_size</code> allows us to uniformly distribute the tasks across the vehicles. Because cuOpt still tries to minimize the number of vehicle, we need to overwite this by adding <code style="background:lightgreen;color:black">min_vehicles</code> in the Fleet Data.

In [ ]:
import math

In [ ]:
orders_per_vehicle = 3
cuopt_problem_data["solver_config"]["objectives"]= {
                              "cost": 0,
                              "travel_time": 0,
                              "variance_route_size":orders_per_vehicle,
                              "variance_route_service_time": 0,
                              "prize": 0,
                              "vehicle_fixed_cost": 0   
                          }
cuopt_problem_data["fleet_data"]["min_vehicles"] = math.ceil(len(task_locations)/orders_per_vehicle)

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]
if "solver_response" in solver_resp:
    solver_resp = solver_resp["solver_response"]
else:
    # For our purposes here if we get an infeasible response,
    # we treat it as a successful solution
    solver_resp = solver_resp["solver_infeasible_response"]
    solver_resp["status"] = 0
    print("Infeasible solution found!")

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

Similarly, <code style="background:lightgreen;color:black">variance_route_service_time</code> allows us to uniformly distribute the service time across the vehicles. We know that install service takes 60 minutes and maintenance service takes 90 minutes. With a higher value, the routes are more venly distributed. 

cuOpt still tries to minimize the number of vehicles used in the solution, which could mean some routes are still longer than others. We can overwrite this by introducing <code style="background:lightgreen;color:black">min_vehicles</code>, where we set the minimum number of vehicles to be ised in the solution. In this case, we want this number to be higher than cuOpt's default response. 


Feel free to play around with these value to see how they affects the solution.

In [ ]:
variance_route_service_time = 300
cuopt_problem_data["solver_config"]["objectives"]= {
                              "cost": 0,
                              "travel_time": 0,
                              "variance_route_size":variance_route_service_time,
                              "variance_route_service_time": 0,
                              "prize": 0,
                              "vehicle_fixed_cost": 0   
}
cuopt_problem_data["fleet_data"]["min_vehicles"] = math.ceil(sum(demand_time)/variance_route_service_time)

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]
if "solver_response" in solver_resp:
    solver_resp = solver_resp["solver_response"]
else:
    # For our purposes here if we get an infeasible response,
    # we treat it as a successful solution
    solver_resp = solver_resp["solver_infeasible_response"]
    solver_resp["status"] = 0
    print("Infeasible solution found!")

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

Finally, let's take a look at <code style="background:lightgreen;color:black">vehicle_fixed_cost</code>. Let's imagine some workers are full time employees and some are contracters. Fulltime workers are already on payroll so there is no additional costs, whereas contract workers must be paid extra. This is meaningful when the number of full time workers are not sufficient and the businesses have to hire contractors fulfill the requirement.  

First, let's delete the min_vehicles we set for the objective function above, since it is not relevant in setting Vehicle Fixed Cost.

Let's start by setting this associated cost for vehicles. We will randomly select half of the drivers to be full time employees with an associated fixed cost of 200, and the other half will be contract workers with an associated fixed cost of 300.

In [ ]:
del cuopt_problem_data["fleet_data"]["min_vehicles"]

In [ ]:
fixed_cost = [200] * int(n_vehicles/2) + [300] * int(n_vehicles/2)
random.shuffle(fixed_cost)

cuopt_problem_data["fleet_data"]["vehicle_fixed_costs"] = fixed_cost

In [ ]:
#saving two lists of full time employees and contractors
full_time_drivers = []
contractors = []
for index, v in enumerate(fixed_cost):
    if v==200:
        full_time_drivers.append(index)
    else:
        contractors.append(index)

Similarly to the other objectives, let's set these values in the solver config section of our data. 

In [ ]:
cuopt_problem_data["solver_config"]["objectives"]= {
                              "cost": 1,
                              "travel_time": 0,
                              "variance_route_size": 0,
                              "variance_route_service_time": 0,
                              "prize": 0,
                              "vehicle_fixed_cost": 500   
}

In [ ]:
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]
if "solver_response" in solver_resp:
    solver_resp = solver_resp["solver_response"]
else:
    # For our purposes here if we get an infeasible response,
    # we treat it as a successful solution
    solver_resp = solver_resp["solver_infeasible_response"]
    solver_resp["status"] = 0
    print("Infeasible solution found!")

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

Let's check how many full time workers vs. how many contractors are dispatched in the solution.

In [ ]:
full_time_count = 0
contractors_count = 0
for v in solver_response['response']['solver_response']['vehicle_data'].keys():
    if int(v) in contractors:
        contractors_count+=1
    else: full_time_count+=1

print("there are {} full time drivers dispatched in the solution".format(full_time_count))
print("there are {} contracted drivers dispatched in the solution".format(contractors_count))

The weight assigned to `vehicle_fixed_cost` in the objectives section of the `data_config` is somehwat arbitrary. The higher that weight is, the more cuOpt prioritizes using vehicles with lower cost. With the current value of 500, there are 3 contractors in the solution. With a value of 100, there are 5 contractors in the solution. PLay around with that value to see how it affects the solution.

## License

SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.